In [1]:
import sqlite3
import os
import glob
import csv

In [2]:
# Função para tratar os valores monetários e converter para o formato DECIMAL
def tratar_valor_monetario(valor):
    return float(valor.replace('.', '').replace(',', '.'))

In [3]:
# Função para criar as tabelas no banco de dados
def criar_tabelas(cursor):
    # Criar a tabela para Saques
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Saques (
        AnoMesReferencia TEXT,
        AnoMesCompetencia TEXT,
        UF TEXT,
        CodigoMunicipioSIAFI INTEGER,
        NomeMunicipioSIAFI TEXT,
        CPFBeneficiario TEXT,
        NISBeneficiario TEXT,
        NomeBeneficiario TEXT,
        DataSaque DATE,
        ValorParcela DECIMAL
    )
    ''')

    # Criar a tabela para Pagamentos
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Pagamentos (
        AnoMesReferencia TEXT,
        AnoMesCompetencia TEXT,
        UF TEXT,
        CodigoMunicipioSIAFI INTEGER,
        NomeMunicipioSIAFI TEXT,
        CPFBeneficiario TEXT,
        NISBeneficiario TEXT,
        NomeBeneficiario TEXT,
        ValorParcela DECIMAL
    )
    ''')

In [4]:
# Função para processar cada arquivo CSV e inserir os dados nas tabelas apropriadas
def processar_e_inserir_arquivos(pasta_raiz, cursor):
    # Lista todas as subpastas seguindo o padrão especificado
    subpastas = glob.glob(os.path.join(pasta_raiz, '2020??_BolsaFamilia_*'))
    
    for subpasta in subpastas:
        # Determinar se a subpasta é para Saques ou Pagamentos
        tipo = 'Saques' if 'Saques' in subpasta else 'Pagamentos'
        
        # Lista todos os arquivos CSV na subpasta
        arquivos_csv = glob.glob(os.path.join(subpasta, '*.csv'))
        
        for arquivo_csv in arquivos_csv:
            with open(arquivo_csv, 'r', encoding='ISO-8859-1') as csvfile:
                csvreader = csv.reader(csvfile, delimiter=';')
                next(csvreader, None)  # Pula o cabeçalho
                
                for row in csvreader:
                    # Ajustar os campos de acordo com o tipo
                    if tipo == 'Saques':
                        AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI, \
                        CPFBeneficiario, NISBeneficiario, NomeBeneficiario, DataSaque, ValorParcela = row
                        
                        DataSaque = DataSaque.split('/')[2] + '-' + DataSaque.split('/')[1] + '-' + DataSaque.split('/')[0]  # Ajustar a data para o formato YYYY-MM-DD
                        ValorParcela = tratar_valor_monetario(ValorParcela)  # Converter o valor para DECIMAL
                        
                        cursor.execute('''
                            INSERT INTO Saques (
                                AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI,
                                CPFBeneficiario, NISBeneficiario, NomeBeneficiario, DataSaque, ValorParcela
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI,
                              CPFBeneficiario, NISBeneficiario, NomeBeneficiario, DataSaque, ValorParcela))
                    else:
                        AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI, \
                        CPFBeneficiario, NISBeneficiario, NomeBeneficiario, ValorParcela = row
                        
                        ValorParcela = tratar_valor_monetario(ValorParcela)  # Converter o valor para DECIMAL
                        
                        cursor.execute('''
                            INSERT INTO Pagamentos (
                                AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI,
                                CPFBeneficiario, NISBeneficiario, NomeBeneficiario, ValorParcela
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI,
                              CPFBeneficiario, NISBeneficiario, NomeBeneficiario, ValorParcela))


In [5]:
# Caminho da pasta raiz onde as subpastas estarão localizadas
pasta_raiz = '/projetos/Mestrado/FBD' 

# Conectar ao banco de dados
conn = sqlite3.connect('BolsaFamilia.db')
cursor = conn.cursor()

# Criar tabelas
criar_tabelas(cursor)


# Chamar a função para processar e inserir os arquivos
processar_e_inserir_arquivos(pasta_raiz, cursor)

# Salvar as alterações e fechar a conexão com o banco de dados
conn.commit()
conn.close()

In [7]:
conn = sqlite3.connect('BolsaFamilia.db')
cursor = conn.cursor()

# Consulta para selecionar os primeiros 5 registros da tabela Saques
cursor.execute('SELECT * FROM Saques LIMIT 10')
saques_samples = cursor.fetchall()

# Consulta para selecionar os primeiros 5 registros da tabela Pagamentos
cursor.execute('SELECT * FROM Pagamentos LIMIT 10')
pagamentos_samples = cursor.fetchall()

# Fechar a conexão após a consulta
conn.close()

# Retornar os resultados das consultas como exemplos
(saques_samples, pagamentos_samples)

([('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.833.642-**',
   '16167611395',
   'ABIGAIL DAGMAR MACHADO',
   '2020-01-24',
   253),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.332.032-**',
   '16120063855',
   'ADALGIZA NOGUEIRA DOS SANTOS',
   '2020-01-27',
   170),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.089.602-**',
   '21208225415',
   'ADEIDE RODRIGUES DAMASCENO',
   '2020-01-27',
   422),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.469.322-**',
   '16036929952',
   'ADELINA XUITES JACOB',
   '2020-01-21',
   457),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.217.922-**',
   '20500011774',
   'ADENILDA TAVARES DA SILVA',
   '2020-02-27',
   297),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.593.192-**',
   '20326549867',
   'ADENILZA MACHADO PINHEIRO',
   '2020-02-06',
   171),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
  